In [6]:
# Model design
import agentpy as ap
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import split, nearest_points, snap
import copy
import geopandas
import momepy

# Visualization
import matplotlib.pyplot as plt 



class Pedestrian(ap.Agent):

    def setup(self):
        # Init random number generator
        rng = self.model.random
        
        # Initialize attributes and variables
        self.walking_speed = rng.random() + 1
        self.metric_path = []
        self.leftover_distance = 0
        
        # Choose random origin within boundaries of Quakenbrueck
        self.orig, self.dest = self.generate_random_orig_dest(self.model.area_polygon, 250)
                
        # Find the closest nodes in the network for origin and destination
        self.orig_node_id = self.get_nearest_node(self.model.nodes, self.orig)
        self.dest_node_id = self.get_nearest_node(self.model.nodes, self.dest)
        
        
        # Set current location to the origin node
        self.location = self.model.nodes.loc[[self.orig_node_id]]
        
        # Compute shortest path to destination
        self.agent_compute_path()

    def generate_random_orig_dest(self, polygon, min_dist):
        """
        Create random origin and destination coordinates inside model polygon boundaries
        with a minimum distance of min_dist (meter) apart.

        Parameters
        ----------
        allowed_polygon : shapely.geometry.polygon.Polygon
            Polygon - sets spatial boundaries to the origin and destination points 
        min_dist : float
            minimum distance between origin and destination

        Returns
        -------
        Point, Point
            Origin and destination as shapely.geometry.point.Point
        """
        # Init random number generator
        rng = self.model.random
        points = []
        minx, miny, maxx, maxy = polygon.bounds.values[0]
        while len(points) < 2:
            pnt = Point(rng.uniform(minx, maxx), rng.uniform(miny, maxy))
            # check if point in allowed area
            if polygon.contains(pnt).values[0]:
                # check if there is a origin already, if so: make sure next point is at least min_dist apart
                if len(points) == 1:
                    distance = points[0].distance(pnt)
                    if distance > min_dist:
                        points.append(pnt)
                else:
                    points.append(pnt)
        return points[0], points[1]        

    def get_nearest_node(self, nodes, point):
        """
            Return the nearest node to a given point from of a set of nodes.

            Parameters
            ----------
            nodes : geopandas.GeoDataFrame
                GeoDataFrame with multiple point geometries 
            point : shapely.geometry.point.Point
                Point  

            Returns
            -------
            Index
                The GeoDataFrame index of the nearest node 
        """
        multipoint = nodes.geometry.unary_union
        queried_geom, nearest_geom = nearest_points(point, multipoint)
        res = nodes.index[nodes['geometry'] == nearest_geom].tolist()[0]
        return res
        
    def agent_compute_path(self):
        """
            Calculate the shortest path from the agents current location to its destination.
            Stores result as list of nodes in a shortest path.
        """
        self.metric_path = nx.dijkstra_path(self.model.G, source=self.location['nodeID'].values[0], target=self.dest_node_id, weight='mm_len')
                
    def get_next_position(self, duration):
        """
            Calculates the position of an agent after the next timestep, dependet on the duration of a timestep and the walking speed of the agent.

            Parameters
            ----------
            duration : float
                The duration of a timestep in seconds.  
        """
        # if path is shorter than 2, destination is reached, return
        if len(self.metric_path) < 2:
            return 
        else:
            # calculate the distance the agent passes by during the current timestep, to prevent agent from walking further than the next node
            walking_distance = self.walking_speed * duration
            current_node = self.location
            next_node = self.model.nodes.loc[[self.metric_path[1]]]
            edge = self.model.G.get_edge_data(self.metric_path[0],self.metric_path[1])
            
            # TODO: check whether copy is necessary or original edge can be used
            current_edge = copy.deepcopy(list(edge.values())[0])
            # check if linestring starts at current node (and ends at next node)
            if(current_edge['geometry'].coords[0] != self.model.nodes.loc[self.model.nodes['nodeID'] == self.metric_path[0]]['geometry'].values[0].coords[0]):
                # invert indice order
                current_edge['geometry'] = LineString(list(current_edge['geometry'].coords)[::-1])
                
            distance_to_next_point = 0
            # check whether pedestrian is on edge (leftover_distance != 0) or on node (leftover_distance == 0) 
            if(self.leftover_distance != 0):
                # TODO: Check if there is a better way to solve issue with current node not being on the edge because of rounded coords
                snapped_edge = snap(current_edge['geometry'], current_node['geometry'].values[0], 0.01)
                current_edge['geometry'] = split(snapped_edge, current_node['geometry'].values[0])[1]
                distance_to_next_point = self.leftover_distance
            else:
                distance_to_next_point = current_edge['mm_len']
                
            # check if pedestrian would walk past next node or if length of vector is 0, in that case:
            if distance_to_next_point < walking_distance:                
                # set current node to location of next node
                self.leftover_distance = 0
                self.metric_path.pop(0)
                self.location = next_node
                self.check_next_street_segment()
                return
            # if next node is not reached go on with calculation of next position
            else:
                new_location = current_edge['geometry'].interpolate(walking_distance)
                self.location['geometry'] = new_location
                self.leftover_distance = distance_to_next_point - walking_distance
                return
    
    def check_next_street_segment(self):
        """
            Check whether the next street segement is too crowded or has an intervention that stops the agent from accessing it.
            If there is an obstacle, recalculates the agents path and overwrites the previous path.
        """
        # TODO: implement this function!
        current_node = self.metric_path[0]
        # next_node = self.metric_path[1]
        # next_edge = self.model.edges.loc[(self.model.edges['u'] == current_node) & (self.model.edges['v'] == next_node)]
        # next_edge_inverse = next_edge = self.model.edges.loc[(self.model.edges['v'] == current_node) & (self.model.edges['u'] == next_node)]
        # if(next_edge['people_count'] > self.people_count):
            
            
        
class MyModel(ap.Model):

    def setup(self):
        # Read boundary polygon and make sure it uses CRS EPSG:3857
        self.area_polygon = geopandas.read_file("./boundaries/crop_area.shp")
        self.area_polygon = self.area_polygon.to_crs('EPSG:3857')
        
        # Read street network as geopackage and convert it to GeoDataFrame
        streets = geopandas.read_file("./network-data/quakenbrueck_clean.gpkg")
        # Transform GeoDataFrame to networkx Graph
        self.G = momepy.gdf_to_nx(streets, approach='primal')
        # Calculate degree of nodes
        self.G = momepy.node_degree(self.G, name='degree')
        # Convert graph back to GeoDataFrames with nodes and edges
        self.nodes, self.edges, sw = momepy.nx_to_gdf(self.G, points=True, lines=True, spatial_weights=True)
        # set index column, and rename nodes in graph 
        self.nodes = self.nodes.set_index("nodeID", drop=False)
        self.nodes = self.nodes.rename_axis([None])
        sorted(self.G)
        mapping = dict(zip([(geom.x, geom.y) for geom in self.nodes['geometry'].tolist()], self.nodes.index[self.nodes['nodeID']].tolist()))
        self.G = nx.relabel_nodes(self.G, mapping)

        # Create a list of agents 
        self.agents = ap.AgentList(self, self.p.agents, Pedestrian)
        
        # Store list of inital paths into global list  
        self.routes = self.agents.metric_path
        
        # opt. visualize network nodes, edges and degree values
        if self.p.viz:
            f, ax = plt.subplots(figsize=(10, 10))
            self.nodes.plot(ax=ax, column='degree', cmap='tab20b', markersize=(2 + self.nodes['nodeID'] * 4), zorder=2)
            self.edges.plot(ax=ax, color='lightgrey', zorder=1)
            ax.set_axis_off()
            plt.show()
                    
    def step(self):
        """ Call a method for every agent. """
        # Calculate next position for all agents 
        self.agents.get_next_position(self.model.p.duration)

    def update(self):
        """ Record a dynamic variable. """
        # TODO: Record agents position as shapefile
        self.agents.record('metric_path')
        self.model.record('G')
        # store all the agents current location in list 
        self.positions = self.agents.location

    def end(self):
        """ Report an evaluation measure. """

# specify some parameters
parameters = {
    'agents': 50,
    'steps': 20,
    'comment': False,
    'viz': False,
    'walking_speed': 1.5,
    'duration': 5
}

# Run the model!
model = MyModel(parameters)
results = model.run()     

Completed: 20 steps
Run time: 0:00:04.919650
Simulation finished


In [3]:
import IPython


In [7]:
def animation_plot(m, ax1):
    ax1.set_title("Pedestrian Movement")
    # Plot network with agents
    m.edges.plot(ax=ax1, figsize=(40,40), color="gray", lw=1.0)
    m.positions.plot(ax=ax1, color="red", markersize=5)

# create empty plot to start
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot()

# specify parameters
parameters = {
    'agents': 50,
    'steps': 20,
    'comment': False,
    'viz': False,
    'walking_speed': 1.5,
    'duration': 5
}
# run model and display animatiun
animation = ap.animate(MyModel(parameters), fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())